In [1]:
# Import packages
%matplotlib inline

import os
import sys
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# Question 4: (30 points) Exploratory Analysis

In [2]:
# Load dataset
bristol_air = pd.read_csv('BristolAir_Exploratory.csv', compression='bz2')

In [3]:
bristol_air.head()

,Date Time,NOx,NO2,NO,SiteID,Loc.Lat,Loc.Long
0,2007-03-27 13:00:00+00:00,87.00,28.00,28.75,1,51.441747,-2.559956
1,2007-03-27 14:00:00+00:00,98.50,32.00,32.50,1,51.441747,-2.559956
2,2007-03-27 20:00:00+00:00,202.00,97.75,68.00,6,51.427864,-2.563742
3,2007-03-28 02:00:00+00:00,107.80,39.52,44.52,8,51.489993,-2.688779
4,2007-03-28 04:00:00+00:00,115.78,31.94,54.67,8,51.489993,-2.688779


## 4.1 (6 points)

In [4]:
print("Dataset shape: {}".format(bristol_air.shape))
print("Number of instances: {}".format(bristol_air.shape[0]))
print("Number of attributes: {}".format(bristol_air.shape[1]))

Dataset shape: (1306758, 7)
Number of instances: 1306758
Number of attributes: 7


In [5]:
bristol_air.describe()

,NOx,NO2,NO,SiteID,Loc.Lat,Loc.Long
count,1.191220e+06,1.188426e+06,1.197536e+06,1.306758e+06,1.306758e+06,1.306758e+06
mean,1.200819e+02,4.493445e+01,4.956930e+01,8.411554e+00,5.145213e+01,-2.580516e+00
std,1.428863e+02,3.183197e+01,7.645375e+01,5.329009e+00,1.607845e-02,2.738316e-02
min,-3.108000e+01,-6.669844e+00,-1.771000e+01,0.000000e+00,5.142786e+01,-2.688779e+00
25%,3.550000e+01,2.200000e+01,7.000000e+00,4.000000e+00,5.144175e+01,-2.592724e+00
50%,7.300000e+01,3.800000e+01,2.200000e+01,7.000000e+00,5.145543e+01,-2.582253e+00
75%,1.450000e+02,5.950000e+01,5.700000e+01,1.400000e+01,5.146067e+01,-2.562714e+00
max,2.164250e+03,5.765000e+02,1.231250e+03,1.700000e+01,5.148999e+01,-2.535230e+00


In [6]:
bristol_air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1306758 entries, 0 to 1306757
Data columns (total 7 columns):
Date Time    1306758 non-null object
NOx          1191220 non-null float64
NO2          1188426 non-null float64
NO           1197536 non-null float64
SiteID       1306758 non-null int64
Loc.Lat      1306758 non-null float64
Loc.Long     1306758 non-null float64
dtypes: float64(5), int64(1), object(1)
memory usage: 69.8+ MB


The Bristol Air Exploratory dataset has 1,306,758 instances and 7 attributes

In [ ]:
bristol_air.head()

## 4.2 (6 points)

## 4.3 (4 points)

## 4.4 (3 points)

## 4.5 (2 points)

## 4.6 (5 points)

## 4.7 (4 points)

# Question 5: (19 points) Principal Component Analysis

## 5.1 (1 point)

## 5.2 (5 points)

## 5.3 (2 points)

## 5.4 (5 points)

## 5.5 (6 points)

# Question 6: (41 points) Regression

## 6.1 (2 points)

## 6.2 (4 points)

## 6.3 (3 points)

## 6.4 (5 points)

## 6.5 (5 points)

## 6.6 (5 points)

## 6.7 (4 points)

## 6.8 (1 point)

## 6.9 (4 points)

## 6.10 (3 points)

## 6.11 (5 points)